# Introduction

F1 is celebrating its 70th edition in 2020. Due to the worldwide pandemic situation, the sport was affected as many others and the season is now in progress with a very unusual calendar. It is evident from the first few races how Mercedes is dominating the competition, a result somewhat expected after winning the last 6 constructor titles.

As the season unfolds, it can be expected some other team will have their day on some tracks but the curiosity about how dominant the team is remains. 

In this notebook, we will look at the historical performance of drivers and constructors to see what *dominant seasons* look like and how 2020, in all its peculiarity, compares to these seasons. This is by no means an attempt of finding who is the best driver in history or similar exercises, but rather observe how, year by year, how a driver or a constructor won the championship changed, making a season more thrilling than another.

Moreover, we will be looking at thrilling races (when we have data about it) and give an idea on how data can capture this aspect of an F1 race

In [ ]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import seaborn as sns
%matplotlib inline

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
pd.set_option('max_columns', 100)

In [ ]:
def import_all():
    data = {}
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            name = filename.replace('.csv', '')
            data[name] = pd.read_csv(os.path.join(dirname, filename))
            
    return data


def add_ids(data, key):
    
    df = data[key]
    n_lines = df.shape[0]

    df = pd.merge(df, data['races'][['raceId', 
                                     'year', 'round', 
                                     'circuitId', 'date', 'time']], 
                  on='raceId', how='left')
    if df.shape[0] != n_lines:
        raise ValueError('Merging raceId went wrong')
        
    df = pd.merge(df, data['circuits'][['circuitId', 
                                        'circuitRef', 'location', 'country']], 
                  on='circuitId', how='left')
    if df.shape[0] != n_lines:
        raise ValueError('Merging circuitId went wrong')
        
    df = pd.merge(df, data['drivers'][['driverId', 
                                       'driverRef', 'forename', 'surname', 
                                       'dob', 'nationality']].rename(columns={'nationality': 'drv_nat'}), 
                  on='driverId', how='left')
    if df.shape[0] != n_lines:
        raise ValueError('Merging driverId went wrong')
    
    if (key != 'lap_times') and (key != 'pit_stops'):
        df = pd.merge(df, data['constructors'][['constructorId', 
                                                'constructorRef', 
                                                'name', 'nationality']].rename(columns={'nationality': 'cstr_nat'}), 
                      on='constructorId', how='left')
        if df.shape[0] != n_lines:
            raise ValueError('Merging constructorId went wrong')
        
    if key == 'results':
        df = pd.merge(df, data['status'], 
                      on='statusId', how='left')
        if df.shape[0] != n_lines:
            raise ValueError('Merging statusId went wrong')
        
    return df

In [ ]:
data = import_all()

res = add_ids(data, 'results')
qual = add_ids(data, 'qualifying')
laps = add_ids(data, 'lap_times')
pits = add_ids(data, 'pit_stops')

laps.rename(columns={'time_x': 'lap_time', 'time_y': 'time'}, inplace=True)
res.rename(columns={'time_x': 'race_time', 'time_y': 'time'}, inplace=True)
pits.rename(columns={'time_x': 'pit_time', 'time_y': 'time'}, inplace=True)

laps = pd.merge(laps, res[['raceId', 'driverId', 
                           'constructorRef', 'name', 'cstr_nat']], 
                on=['raceId', 'driverId'], how='left')
pits = pd.merge(pits, res[['raceId', 'driverId', 
                           'constructorRef', 'name', 'cstr_nat']], 
                on=['raceId', 'driverId'], how='left')

In [ ]:
res[['lap_mins', 'lap_secs']] = res['fastestLapTime'].str.split(':', expand=True)
res[['lap_secs', 'lap_millisecs']] = res['lap_secs'].str.split('.', expand=True)
res['lap_mins'] = pd.to_numeric(res['lap_mins'], errors='coerce').fillna(99)
res['lap_secs'] = pd.to_numeric(res['lap_secs'], errors='coerce').fillna(99)
res['lap_millisecs'] = pd.to_numeric(res['lap_millisecs'], errors='coerce').fillna(99)

res['fastestLapTime_ms'] = (60 * res['lap_mins'] + res['lap_secs']) * 1000 + res['lap_millisecs']

res['race_fastestTime'] = res.groupby('raceId').fastestLapTime_ms.transform('min')
res['FastLap'] = np.where(res['race_fastestTime'] == res['fastestLapTime_ms'], 1, 0)

res.drop(['lap_mins', 'lap_secs', 'lap_millisecs'], axis=1, inplace=True)

points = {1: 25, 2: 18, 3: 15, 4: 12, 5: 10, 6: 8, 7: 6, 8: 4, 9: 2, 10: 1}

res['points'] = res['positionOrder'].map(points).fillna(0)
#res.loc[res.FastLap == 1, 'points'] = res['points'] + 1

res['fastestLap'] = pd.to_numeric(res['fastestLap'], errors='coerce')

res['DriverName'] = res['forename'].str[0] + '. ' + res['surname']


res['net_gain'] = -(res['positionOrder'] - res['grid'])
res['abs_gain'] = abs(res['net_gain'])

res['finished'] = np.where(res.status == 'Finished', 1, 0)

In [ ]:
def plot_frame(ax):
    ax.set_facecolor('#292525')
    ax.spines['bottom'].set_color('w')
    ax.tick_params(axis='x', colors='w')
    ax.xaxis.label.set_color('w')
    ax.spines['left'].set_color('w')
    ax.tick_params(axis='y', colors='w')
    ax.yaxis.label.set_color('w')
    return ax

def get_drv_ann(data, year, ax, adjust, count=False, measure='Pts.'):
    
    yr_data = data[data.year==year].groupby(['driverId', 'DriverName', 'name']).points.sum().sort_values(ascending=False)
    if count:
        yr_data = data[data.year==year].groupby(['driverId', 'DriverName', 'name']).resultId.count().sort_values(ascending=False)
    drv_name = yr_data.index[0][1]
    ctr_name = yr_data.index[0][2]
    pts = yr_data[0]
    value = pts / data[data.year==year].raceId.nunique()
    
    text = f'{drv_name}\n{ctr_name}, {year}\n{int(pts)} {measure}'
    
    ax.annotate(text, xy=(year, value), xycoords='data', xytext=adjust, textcoords='offset points', color='w')
    
    return ax


def get_ctr_ann(data, year, ax, adjust, count=False, measure='Pts.'):
    
    yr_data = data[data.year==year].groupby(['name']).points.sum().sort_values(ascending=False)
    if count:
        yr_data = data[data.year==year].groupby(['name']).resultId.count().sort_values(ascending=False)
    ctr_name = yr_data.index[0]
    pts = yr_data[0]
    value = pts / data[data.year==year].raceId.nunique()
    
    text = f'{ctr_name}, {year}\n{int(pts)} {measure}'
    
    ax.annotate(text, xy=(year, value), xycoords='data', xytext=adjust, textcoords='offset points', color='w')
    
    return ax


def plot_bars(bars, ax, color):
    
    colors = [color if (c == 2020) else 'w' for c in bars.index]
    bars.plot(color=colors, kind='bar', ax=ax)
    ax.set_title('Top Years vs 2020', fontsize=14, color='w')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=0)
    
    return ax

# The best drivers and the best cars

To compare cars and drivers of different eras we have to take into account a few things:
* The point system changed
* Some years the fastest lap was awarded one extra point, thus giving one more reason to aim for it
* Each season has a different number of races

Therefore, we assigned all the points with the current system (25 to the winner, 18 to p2, etc) and consider some statistics by weighting them on the number of Grand Prix of each season. Since the lap time was not recorded consistently across the years, we ignore the extra point for the fastest lap.

The 2020 season has been already a very peculiar one due to reasons outside of the F1 control. However, we will see how the Mercedes dominance in 2020 is somewhat unprecedented in the history of the sport.

In [ ]:
fig = plt.figure(figsize=(15, 25), facecolor='#292525')
fig.subplots_adjust(top=0.95)
fig.suptitle('Best Driver of the Season', fontsize=18, color='w')

gs = GridSpec(5, 3, figure=fig)
ax0 = fig.add_subplot(gs[0, :2])
ax1 = fig.add_subplot(gs[0, 2])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1:])
ax4 = fig.add_subplot(gs[2, :2])
ax5 = fig.add_subplot(gs[2, 2])
ax6 = fig.add_subplot(gs[3, 0])
ax7 = fig.add_subplot(gs[3, 1:])
ax8 = fig.add_subplot(gs[4, :2])
ax9 = fig.add_subplot(gs[4, 2])

race_counts = res.groupby(['year']).raceId.nunique()

(res.groupby(['year', 'driverId']).points.sum().groupby('year').max() / race_counts).plot(ax=ax0, color='#15E498')
ax0.set_title('Most Points', fontsize=14, color='w')
ax0.set_ylabel('Points per GP', fontsize=12)
ax0 = get_drv_ann(res, 2002, ax0, (-80, -25))
ax0 = get_drv_ann(res, 1963, ax0, (-50, -2))
ax0 = get_drv_ann(res, 1970, ax0, (-90, 2))
ax0 = get_drv_ann(res, 1982, ax0, (6, 2))

years = (res.groupby(['year', 'driverId']).points.sum().groupby('year').max() / race_counts).sort_values()[-6:].index.to_list()
if 2020 not in years:
    years += [2020]
bars = (res[res.year.isin(years)].groupby(['year', 'DriverName']).points.sum().groupby(['year']).max() / race_counts).dropna()
ax1 = plot_bars(bars, ax1, '#15E498')

(res[res.positionOrder == 1].groupby(['year', 'driverId']).resultId.count().groupby('year').max() / race_counts).plot(ax=ax3, color='#C3C92E')
ax3.set_title('Most Wins', fontsize=14, color='w')
ax2.set_ylabel('Wins per GP', fontsize=12)
ax3 = get_drv_ann(res[res.positionOrder == 1], 1952, ax3, (5, -22), count=True, measure='Wins')
ax3 = get_drv_ann(res[res.positionOrder == 1], 2004, ax3, (-80, -25), count=True, measure='Wins')
ax3 = get_drv_ann(res[res.positionOrder == 1], 1982, ax3, (10, 0), count=True, measure='Wins')
ax3 = get_drv_ann(res[res.positionOrder == 1], 2012, ax3, (-20, -40), count=True, measure='Wins')

years = (res[res.positionOrder == 1].groupby(['year', 'driverId']).resultId.count().groupby('year').max() / race_counts).sort_values()[-6:].index.to_list()
if 2020 not in years:
    years += [2020]
bars = (res[(res.positionOrder == 1) & (res.year.isin(years))].groupby(['year', 'DriverName']).resultId.count().groupby(['year']).max() / race_counts).dropna()
ax2 = plot_bars(bars, ax2, '#C3C92E')

(res[res.positionOrder <= 3].groupby(['year', 'driverId']).resultId.count().groupby('year').max() / race_counts).plot(ax=ax4, color='#C93D2E')
ax4.set_title('Most Podiums', fontsize=14, color='w')
ax4.set_ylabel('Podiums per GP', fontsize=12)
ax4 = get_drv_ann(res[res.positionOrder <= 3], 2002, ax4, (-80, -25), count=True, measure='Podiums')
ax4 = get_drv_ann(res[res.positionOrder <= 3], 1963, ax4, (5, 0), count=True, measure='Podiums')
ax4 = get_drv_ann(res[res.positionOrder <= 3], 1982, ax4, (10, 0), count=True, measure='Podiums')
ax4 = get_drv_ann(res[res.positionOrder <= 3], 1970, ax4, (-100, 0), count=True, measure='Podiums')

years = (res[res.positionOrder <= 3].groupby(['year', 'driverId']).resultId.count().groupby('year').max() / race_counts).sort_values()[-6:].index.to_list()
if 2020 not in years:
    years += [2020]
bars = (res[(res.positionOrder <= 3) & (res.year.isin(years))].groupby(['year', 'DriverName']).resultId.count().groupby(['year']).max() / race_counts).dropna()
ax5 = plot_bars(bars, ax5, '#C93D2E')

(res[res.grid == 1].groupby(['year', 'driverId']).resultId.count().groupby('year').max() / race_counts).plot(ax=ax7, color='#3A3FDC')
ax7.set_title('Most Pole Positions', fontsize=14, color='w')
ax6.set_ylabel('Poles per GP', fontsize=12)
ax7 = get_drv_ann(res[res.grid == 1], 2011, ax7, (10, -10), count=True, measure='Poles')
ax7 = get_drv_ann(res[res.grid == 1], 2009, ax7, (10, -10), count=True, measure='Poles')
ax7 = get_drv_ann(res[res.grid == 1], 1992, ax7, (20, -23), count=True, measure='Poles')
ax7 = get_drv_ann(res[res.grid == 1], 1980, ax7, (25, -5), count=True, measure='Poles')

years = (res[res.grid == 1].groupby(['year', 'driverId']).resultId.count().groupby('year').max() / race_counts).sort_values()[-6:].index.to_list()
if 2020 not in years:
    years += [2020]
bars = (res[(res.grid == 1) & (res.year.isin(years))].groupby(['year', 'DriverName']).resultId.count().groupby(['year']).max() / race_counts).dropna()
ax6 = plot_bars(bars, ax6, '#3A3FDC')

(res[res['rank'] == '1'].groupby(['year', 'driverId']).resultId.count().groupby('year').max() / race_counts).plot(ax=ax8, color='#41DA5B')
ax8.set_title('Most Fast Laps', fontsize=14, color='w')
ax8.set_ylabel('Fast Laps per GP', fontsize=12)
ax8 = get_drv_ann(res[res['rank'] == '1'], 2009, ax8, (-80, 0), count=True, measure='FL')
ax8 = get_drv_ann(res[res['rank'] == '1'], 2008, ax8, (10, -30), count=True, measure='FL')
ax8 = get_drv_ann(res[res['rank'] == '1'], 2015, ax8, (-20, 0), count=True, measure='FL')

years = (res[res['rank'] == '1'].groupby(['year', 'driverId']).resultId.count().groupby('year').max() / race_counts).dropna().sort_values()[-6:].index.to_list()
if 2020 not in years:
    years += [2020]
bars = (res[(res['rank'] == '1') & (res.year.isin(years))].groupby(['year', 'DriverName']).resultId.count().groupby(['year']).max() / race_counts).dropna()
ax9 = plot_bars(bars, ax9, '#41DA5B')

for ax in [ax0, ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9]:
    ax = plot_frame(ax)
    ax.set_xlabel('')

plt.show()

In these plots, we see how the best driver in each statistic performed year by year. For example, in 1952 the driver that won the most Grand Prix was Ascari with 6 wins and his percentage of victories in one season is still unbeaten. Moreover, we observe that

* The 2002 and 2004 seasons were both golden seasons for **Michael Schumacher** and its **Ferrari**, collecting an unprecedented number of points, podiums, and victories. He even beats the legendary 1963 season of **Jim Clark** in Lotus. 
* On the other hand, the 1982 season is the one won by the driver that achieved the lowest number of points, podiums, and victories per GP. We will see later how peculiar such season was.
* In qualify, the absolute dominance of Nigel Mansel and its Williams in 1992 and of Sebastian Vettel in Red Bull in 2011 is evident, with 14 and 15 pole positions. Under this aspect, the 2020 season has not been exceptional, given the rivalry between the Mercedes drivers.
* On the other hand, Sebastian Vettel was the driver with most poles in 2009 but also one of the drivers that achieved the least pole positions among the top drivers of each year.

Another way to read the charts above is considering that **the higher the top driver of a season ranks in those statistics, the more he dominated the competition that year**. And the lower a driver ranks, the bigger (or the closer) the competition was that year.

2020 has been so far a pretty uncompetitive season, which is evident to anyone watching the Gran Prix and the dominance Lewis Hamilton is showing in terms of points, podiums, and fastest laps.

If we have a look at the constructors.

In [ ]:
fig = plt.figure(figsize=(15, 25), facecolor='#292525')
fig.subplots_adjust(top=0.95)
fig.suptitle('Best Constructor of the Season', fontsize=18, color='w')

gs = GridSpec(5, 3, figure=fig)
ax0 = fig.add_subplot(gs[0, :2])
ax1 = fig.add_subplot(gs[0, 2])
ax2 = fig.add_subplot(gs[1, 0])
ax3 = fig.add_subplot(gs[1, 1:])
ax4 = fig.add_subplot(gs[2, :2])
ax5 = fig.add_subplot(gs[2, 2])
ax6 = fig.add_subplot(gs[3, 0])
ax7 = fig.add_subplot(gs[3, 1:])
ax8 = fig.add_subplot(gs[4, :2])
ax9 = fig.add_subplot(gs[4, 2])

race_counts = res.groupby(['year']).raceId.nunique()

(res.groupby(['year', 'constructorId']).points.sum().groupby('year').max() / race_counts).plot(ax=ax0, color='#15E498')
ax0.set_title('Most Points', fontsize=14, color='w')
ax0.set_ylabel('Points per GP', fontsize=12)
ax0 = get_ctr_ann(res, 1952, ax0, (10, -10))
ax0 = get_ctr_ann(res, 1982, ax0, (10, -10))
ax0 = get_ctr_ann(res, 2015, ax0, (-30, 10))
ax0 = get_ctr_ann(res, 1988, ax0, (-30, 10))
ax0 = get_ctr_ann(res, 2012, ax0, (-30, -20))

years = (res.groupby(['year', 'constructorId']).points.sum().groupby('year').max() / race_counts).sort_values()[-6:].index.to_list()
if 2020 not in years:
    years += [2020]
bars = (res[res.year.isin(years)].groupby(['year', 'name']).points.sum().groupby(['year']).max() / race_counts).dropna()
ax1 = plot_bars(bars, ax1, '#15E498')

(res[res.positionOrder == 1].groupby(['year', 'constructorId']).resultId.count().groupby('year').max() / race_counts).plot(ax=ax3, color='#C3C92E')
ax3.set_title('Most Wins', fontsize=14, color='w')
ax2.set_ylabel('Wins per GP', fontsize=12)
ax3 = get_ctr_ann(res[res.positionOrder == 1], 1988, ax3, (-50,-10), count=True, measure='Wins')
ax3 = get_ctr_ann(res[res.positionOrder == 1], 2016, ax3, (-60,0), count=True, measure='Wins')
ax3 = get_ctr_ann(res[res.positionOrder == 1], 1982, ax3, (15,0), count=True, measure='Wins')
ax3 = get_ctr_ann(res[res.positionOrder == 1], 2012, ax3, (-50,-30), count=True, measure='Wins')

years = (res[res.positionOrder == 1].groupby(['year', 'constructorId']).resultId.count().groupby('year').max() / race_counts).sort_values()[-6:].index.to_list()
if 2020 not in years:
    years += [2020]
bars = (res[(res.positionOrder == 1) & (res.year.isin(years))].groupby(['year', 'name']).resultId.count().groupby(['year']).max() / race_counts).dropna()
ax2 = plot_bars(bars, ax2, '#C3C92E')

(res[res.positionOrder <= 3].groupby(['year', 'constructorId']).resultId.count().groupby('year').max() / race_counts).plot(ax=ax4, color='#C93D2E')
ax4.set_title('Most Podiums', fontsize=14, color='w')
ax4.set_ylabel('Podiums per GP', fontsize=12)
ax4 = get_ctr_ann(res[res.positionOrder <= 3], 1952, ax4, (10,-10), count=True, measure='Podiums')
ax4 = get_ctr_ann(res[res.positionOrder <= 3], 1961, ax4, (10,0), count=True, measure='Podiums')
ax4 = get_ctr_ann(res[res.positionOrder <= 3], 1982, ax4, (30,0), count=True, measure='Podiums')
ax4 = get_ctr_ann(res[res.positionOrder <= 3], 2012, ax4, (10,-15), count=True, measure='Podiums')
ax4 = get_ctr_ann(res[res.positionOrder <= 3], 2004, ax4, (-30,5), count=True, measure='Podiums')
ax4 = get_ctr_ann(res[res.positionOrder <= 3], 2015, ax4, (-20,10), count=True, measure='Podiums')

years = (res[res.positionOrder <= 3].groupby(['year', 'constructorId']).resultId.count().groupby('year').max() / race_counts).sort_values()[-6:].index.to_list()
if 2020 not in years:
    years += [2020]
bars = (res[(res.positionOrder <= 3) & (res.year.isin(years))].groupby(['year', 'name']).resultId.count().groupby(['year']).max() / race_counts).dropna()
ax5 = plot_bars(bars, ax5, '#C93D2E')

(res[res.grid == 1].groupby(['year', 'constructorId']).resultId.count().groupby('year').max() / race_counts).plot(ax=ax7, color='#3A3FDC')
ax7.set_title('Most Pole Positions', fontsize=14, color='w')
ax6.set_ylabel('Poles per GP', fontsize=12)
ax7 = get_ctr_ann(res[res.grid == 1], 2009, ax7, (10,0), count=True, measure='Poles')
ax7 = get_ctr_ann(res[res.grid == 1], 1972, ax7, (10,-10), count=True, measure='Poles')
ax7 = get_ctr_ann(res[res.grid == 1], 1956, ax7, (5,-30), count=True, measure='Poles')
ax7 = get_ctr_ann(res[res.grid == 1], 2016, ax7, (-80,0), count=True, measure='Poles')

years = (res[res.grid == 1].groupby(['year', 'constructorId']).resultId.count().groupby('year').max() / race_counts).sort_values()[-6:].index.to_list()
if 2020 not in years:
    years += [2020]
bars = (res[(res.grid == 1) & (res.year.isin(years))].groupby(['year', 'name']).resultId.count().groupby(['year']).max() / race_counts).dropna()
ax6 = plot_bars(bars, ax6, '#3A3FDC')

(res[res['rank'] == '1'].groupby(['year', 'constructorId']).resultId.count().groupby('year').max() / race_counts).plot(ax=ax8, color='#41DA5B')
ax8.set_title('Most Fast Laps', fontsize=14, color='w')
ax8.set_ylabel('Fast Laps per GP', fontsize=12)
ax8 = get_ctr_ann(res[res['rank'] == '1'], 2010, ax8, (10,-10), count=True, measure='FL')
ax8 = get_ctr_ann(res[res['rank'] == '1'], 2012, ax8, (20,0), count=True, measure='FL')
ax8 = get_ctr_ann(res[res['rank'] == '1'], 2004, ax8, (-10,5), count=True, measure='FL')
ax8 = get_ctr_ann(res[res['rank'] == '1'], 2008, ax8, (-10,5), count=True, measure='FL')
ax8 = get_ctr_ann(res[res['rank'] == '1'], 2015, ax8, (-20,5), count=True, measure='FL')

years = (res[res['rank'] == '1'].groupby(['year', 'constructorId']).resultId.count().groupby('year').max() / race_counts).dropna().sort_values()[-6:].index.to_list()
if 2020 not in years:
    years += [2020]
bars = (res[(res['rank'] == '1') & (res.year.isin(years))].groupby(['year', 'name']).resultId.count().groupby(['year']).max() / race_counts).dropna()
ax9 = plot_bars(bars, ax9, '#41DA5B')

for ax in [ax0, ax1, ax2, ax3, ax4, ax5, ax6, ax7, ax8, ax9]:
    ax = plot_frame(ax)
    ax.set_xlabel('')

plt.show()

* Mercedes in 2020 is showing a dominance that was **unseen since the 1960's**. This is actually a clear trend since the past few years, seasons where they were able to achieve up to 90% of the victories (2016, 19 wins). 
* The 2018 season is an exception, with Ferrari getting pretty close and making the season more thrilling.
* Worth mentioning the dominant performances of **1988 McLaren** and **2004 Ferrari**
* On the other hand, the constructor race was pretty close in 2012, the most competitive year in recent history
* 1982 shows once again how competitive the season was.
* **Mercedes is taking every pole position in 2020**, matching the performance of Ferrari in 1956, when they took pole for every one of the 8 (!!) races of the season.


# Season by season

With these first insights, we can have a look at how the mentioned seasons unfolded for the driver and constructor championships. Please remember that **the points are calculated on the current rules, not necessarily the rules used in the given season**.

In [ ]:
def annotate_season(data, ax, tp='Driver'):
    
    text = f'Top {tp} \n'
    for name, points in data.iteritems():
        text += f'{name} {(int(points))}\n'
    
    ax.text(0.05, 0.65, text, transform=ax.transAxes, color='w', fontsize=13)
    
    return ax


def plot_season(res, year, n_drivers, n_ctr, drv_colors, ctr_colors):
    fig, ax = plt.subplots(2, 1, figsize=(15, 10), facecolor='#292525')
    fig.subplots_adjust(top=0.92)
    fig.suptitle(f'{year} Season', fontsize=18, color='w')
    
    drivers = res[res.year==year].groupby('DriverName').points.sum().sort_values()[-n_drivers:].index

    tmp = res[(res.year==year)].sort_values(by=['round'])
    
    to_print = tmp.groupby('DriverName').points.sum().sort_values(ascending=False)[:n_drivers]

    tmp['tot_points'] = tmp.groupby('driverId').points.cumsum()
    tmp = tmp.set_index('raceId')
    i = 0
    for driver in tmp.DriverName.unique():
        if driver in drivers:
            color = drv_colors[i]
            i += 1
        else:
            color = '#7C7373'
        tmp[tmp.DriverName == driver].tot_points.plot(ax=ax[0], color=color)
        
    ax[0] = annotate_season(to_print, ax[0])
           
    constr = res[res.year==year].groupby('name').points.sum().sort_values()[-n_ctr:].index

    tmp = res[(res.year==year)].groupby(['raceId', 'constructorId', 'name'], as_index=False).points.sum().sort_values(by=['raceId'])
    
    to_print = tmp.groupby('name').points.sum().sort_values(ascending=False)[:n_ctr]

    tmp['tot_points'] = tmp.groupby('constructorId').points.cumsum()
    tmp = tmp.set_index('raceId')
    i = 0
    for ctr in tmp.name.unique():
        if ctr in constr:
            color = ctr_colors[i]
            i += 1
        else:
            color = '#7C7373'
        tmp[tmp.name == ctr].tot_points.plot(ax=ax[1], color=color)
        
    ax[1] = annotate_season(to_print, ax[1], tp='Constructor')
        
    ax[0].set_title('Driver Title', fontsize=14, color='w')
    ax[1].set_title('Constructor Title', fontsize=14, color='w')
    
    for axes in ax:
        axes = plot_frame(axes)
        axes.set_xlabel('')
        axes.set_xticks([])
        
    plt.show()
    
    
clrs = {'Ferrari': '#ff2800', 
        'Red Bull': 'b', 
        'McLaren': '#f98e1d', 
        'Williams': 'w', 
        'Mercedes': '#00D2BE', 
        'Brawn': '#B8FD6E', 
        'Lotus': '#FFB800', 
        'BRM': 'g'}

First, let's have a look at 1963 season

In [ ]:
plot_season(res, 1963, n_drivers=3, n_ctr=2, 
            drv_colors=[clrs['BRM'], clrs['BRM'], clrs['Lotus']], 
            ctr_colors=[clrs['BRM'], clrs['Lotus']])

This was a legendary season for Jim Clark, winning in his Lotus 70% of the races, finishing on the podium 90% of the times. An absolutely dominant season. On the other hand, the 2 BRM drivers created quite a challenge for the constructor title, finishing on the podium in every race that year. If the victory of Jim Clark was evident since the early races, we had to wait until the very last race to see Lotus taking the title by only 1 point.

We move forward a couple of decades and find the above mentioned 1982 season.

In [ ]:
plot_season(res, 1982, n_drivers=3, n_ctr=3, 
            drv_colors=[clrs['Ferrari'], clrs['Williams'], clrs['McLaren']], 
            ctr_colors=[clrs['McLaren'], clrs['Ferrari'], clrs['Williams']])

This season started with the drivers going on a strike 3 days before the very first race. Then another protest led by Bernie Ecclestone produced a San Marino Grand Prix with very few cars racing, but also with a great battle between the 2 Ferrari of Didier Pironi and Gilles Villeneuve. Tragically, during free practice of the very next Grand Prix in Belgium, Villeneuve lost his life in a crash. That Gran Prix was the beginning of a quite consistent series of good results, getting with his McLaren on the podium 3 times in 4 races. However, after the Canadian Gran Prix he struggled in getting his car in the point zone. 

After the tragic weekend in Belgium, Ferrari driver Didier Pironi got on the podium 5 times out of 6 races but a big incident during practice at the German Grand Prix ended his F1 career.

This hectic season was eventually won by Keke Rosberg in the Williams, in one of the most uncertain and tragic seasons in F1 history.

A few years later, we can again observe a dominant season. It is 1988 and Prost and Senna are driving for McLaren.

In [ ]:
plot_season(res, 1988, n_drivers=2, n_ctr=1, 
            drv_colors=[clrs['McLaren'], clrs['McLaren']], 
            ctr_colors=[clrs['McLaren']])

McLaren won 93% of the races in 1988, 50% for Senna, 43% for Prost. This produced one of the most legendary rivarlies in F1 history, where the 2 drivers went head to head until the very end while the rest of the field was pretty much a spectator, unable to compete.

Even more extreme was 2002 season.

In [ ]:
plot_season(res, 2002, n_drivers=2, n_ctr=1, 
            drv_colors=[clrs['Ferrari'], clrs['Ferrari']], 
            ctr_colors=[clrs['Ferrari']])

Ferrari won *only* 88% of the races, but Michael Schumacher was on the podium every race that year and on the highest step of the podium 63% of the time. Seasons like this one or the 2004 season led to a series of regulation changes and Ferrari was never able to reproduce that level of dominance. New players came into the picture.

For example, in 2009 we see the triumphant debut of Brawn GP.

In [ ]:
plot_season(res, 2009, n_drivers=4, n_ctr=2, 
            drv_colors=[clrs['Brawn'], clrs['Red Bull'], clrs['Red Bull'], clrs['Brawn']], 
            ctr_colors=[clrs['Red Bull'], clrs['Brawn']])

A technical advantage gave the rookie team a 6-win streak and a substantial early advantage that the main rival Red Bull was unable to close.

Red Bull did not have to wait long to start winning championships but the way those championships were achieved changes a lot year by year. For example, in 2012 we see a thrilling battle between the Ferrari of Fernando Alonso and the Red Bull of Sebastian Vettel

In [ ]:
plot_season(res, 2012, n_drivers=2, n_ctr=3, 
            drv_colors=[clrs['Ferrari'], clrs['Red Bull']], 
            ctr_colors=[clrs['McLaren'], clrs['Red Bull'], clrs['Ferrari']])

Despite Alonso being more consistent (on the podium 65% of the times vs the 50% of Vettel), it was Vettel to get the title at the end. On the constructor side, we can also see how McLaren (with a young Lewis Hamilton) threw its hat in the ring and challenged Ferrari for the second spot in the championship.

On the other hand, only one year later showed how unbeatable Vettel and Red Bull were

In [ ]:
plot_season(res, 2013, n_drivers=2, n_ctr=3, 
            drv_colors=[clrs['Red Bull'], clrs['Ferrari']], 
            ctr_colors=[clrs['Ferrari'], clrs['Red Bull'], clrs['Mercedes']])

Vettel went to the podium 84% of the times and won 64% of the races, the rivalry with Alonso was unfortunately not seen again.

After the Red Bull era, the Mercedes one began and it is still in progress. In particular, the 2015 season showed how big of a difference there was with the rest of the field, winning 84% of the races with the 2 superstars Lewis Hamilton and Nico Rosberg

In [ ]:
plot_season(res, 2015, n_drivers=3, n_ctr=2, 
            drv_colors=[clrs['Mercedes'], clrs['Ferrari'], clrs['Mercedes']], 
            ctr_colors=[clrs['Ferrari'], clrs['Mercedes']])

Before moving on, let's have a look at how the 2020 season is unfolding

In [ ]:
plot_season(res, 2020, n_drivers=3, n_ctr=2, 
            drv_colors=[clrs['Mercedes'], clrs['Red Bull'], clrs['Mercedes']], 
            ctr_colors=[clrs['Red Bull'], clrs['Mercedes']])

# The Best Grand Prix

We can define the net number of overtakes of a driver as the difference between starting and finishing positions. For how limited, this can give us a measure of how thrilling a race was.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7), facecolor='#292525')
fig.suptitle(f'Position changes per Grand Prix', fontsize=18, color='w')

(res.groupby(['year', 'circuitRef']).abs_gain.sum() / res.groupby(['year', 'circuitRef']).size()).groupby('year').mean().plot(label='Mean', color='w')
(res.groupby(['year', 'circuitRef']).abs_gain.sum() / res.groupby(['year', 'circuitRef']).size()).groupby('year').min().plot(label='Min', color='g')
(res.groupby(['year', 'circuitRef']).abs_gain.sum() / res.groupby(['year', 'circuitRef']).size()).groupby('year').max().plot(label='Max', color='r')

leg = ax.legend(facecolor="#292525")
for text in leg.get_texts():
    text.set_color("w")

ax = plot_frame(ax)

We can see how 1989 was a season with a lot of position changes and how after the mid-90s the sport pretty much settled to a relatively low value, where even the most thrilling (under this definition) Grand Prix of each year failed in reaching the levels of the 70s and 80s.

On the other hand, if we consider the proportion of drivers that finished the race with the same number of laps as the race leader, we can see both how the cars became more reliable and how the difference between the top teams and the rest got smaller, on average.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7), facecolor='#292525')
fig.suptitle(f'Proportion of Drivers that finished the race per Grand Prix', fontsize=18, color='w')

res.groupby(['year', 'circuitRef']).finished.mean().groupby('year').mean().plot(color='w', label='Mean')
res.groupby(['year', 'circuitRef']).finished.mean().groupby('year').max().plot(color='r', label='Max')
res.groupby(['year', 'circuitRef']).finished.mean().groupby('year').min().plot(color='g', label='Min')

leg = ax.legend(facecolor="#292525")
for text in leg.get_texts():
    text.set_color("w")

ax = plot_frame(ax)

If we look at the races more in detail, we can see the **proportion of laps in each race when a position change happened** (either via overtake, accident, mistake, or pit stop). Unfortunately, we have only have data starting in 1996, which is the beginning of the negative trend seen in the plots above.

However, we notice that in 2011,  which is also the year of the introduction of the **Drag Reduction System (DRS)**, we have a significant increase in the proportion of laps with at least a position change, going from the 51% of 2010 to the 73% of 2011.

In [ ]:
laps = laps.sort_values(by=['raceId', 'driverId', 'lap'])
laps['pos_change'] = -laps.groupby(['raceId', 'driverId']).position.diff().fillna(0)
laps['abs_change'] = abs(laps['pos_change'])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7), facecolor='#292525')
fig.suptitle(f'Proportion of Laps with at least a position change', fontsize=18, color='w')

tmp = laps.groupby(['year', 'raceId', 'circuitRef', 'lap'], as_index=False).abs_change.sum()
tmp['lap_with_change'] = np.sign(tmp.abs_change)

tmp.groupby(['year', 'raceId', 'circuitRef']).lap_with_change.mean().groupby('year').agg(['mean', 'max', 'min']).plot(ax=ax, color=['w', 'r', 'g'])

leg = ax.legend(facecolor="#292525")
leg.get_texts()[0].set_text('Mean')
leg.get_texts()[1].set_text('Max')
leg.get_texts()[2].set_text('Min')
for text in leg.get_texts():
    text.set_color("w")
    
text = f'Indianapolis \n2005 \n8% of laps'
ax.annotate(text, xy=(2005, 0.08), xycoords='data', xytext=(20,-20), textcoords='offset points', color='w')
text = f'Bahrain \n2013 \n91% of laps'
ax.annotate(text, xy=(2013, 0.91), xycoords='data', xytext=(10,-3), textcoords='offset points', color='w')

ax.set_ylim((0,1))    
ax = plot_frame(ax)

We can also easily spot the most boring race in recent history: Indianapolis 2005. Only 8% of the laps with a position change and expectedly so: due to a problem with the tyres made by Michelin, **only 6 cars were allowed to race that day**.

On the other extreme, we find Bahrain 2013, with **91% of laps with at least a position change**.

Clearly, if we focus only on the top 3 positions of each lap, the numbers go down quite a bit, with the 2011 Belgian Grand Prix being the most exciting race under this statistic.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15, 7), facecolor='#292525')
fig.suptitle(f'Proportion of Laps with at least a position change in the top 3', fontsize=18, color='w')

tmp = laps[laps.position <= 3].groupby(['year', 'raceId', 'circuitRef', 'lap'], as_index=False).abs_change.sum()
tmp['lap_with_change'] = np.sign(tmp.abs_change)

tmp.groupby(['year', 'raceId', 'circuitRef']).lap_with_change.mean().groupby('year').agg(['mean', 'max', 'min']).plot(ax=ax, color=['w', 'r', 'g'])

leg = ax.legend(facecolor="#292525")
leg.get_texts()[0].set_text('Mean')
leg.get_texts()[1].set_text('Max')
leg.get_texts()[2].set_text('Min')
for text in leg.get_texts():
    text.set_color("w")
    
text = f'Belgium \n2011 \n43% of laps'
ax.annotate(text, xy=(2011, 0.43), xycoords='data', xytext=(-30,20), textcoords='offset points', color='w')


ax.set_ylim((0,1))    
ax = plot_frame(ax)

In all of this, 2020 is not being a particularly bad year, if we do not consider the position changes in the top 3. The close competition in the mid field is making most races very enjoyable.

# Race by Race

Following what [James Trotman](https://www.kaggle.com/jtrotman) made in their notebooks (see for example [this one](https://www.kaggle.com/jtrotman/f1-race-traces-2018)), we can now see some of these remarkable races.

According to the previous plot, the most exciting race involving the 3 leaders of a Grand Prix was during the 2011 Belgian Gran Prix. Here, we see at least one position change in the top 3 in 43% of the laps.

In [ ]:
laps['seconds'] = laps['milliseconds'] / 1000
laps['DriverName'] = laps['forename'].str[0] + '. ' + laps['surname']

clrs = {'Ferrari': '#ff2800', 
        'Red Bull': 'b', 
        'McLaren': '#f98e1d', 
        'Williams': 'w', 
        'Mercedes': '#00D2BE', 
        'Brawn': '#B8FD6E', 
        'Lotus': '#FFB800', 
        'BRM': 'g', 
        'Jordan': '#F9D71C'}

def plot_race(data, country, year, colors=None, raceid=None):
    fig, ax = plt.subplots(1, 1, figsize=(13, 10), facecolor='#292525')
    fig.subplots_adjust(top=0.94)
    if raceid is None:
        tmp = data[(data.country == country) & (data.year == year)]
    else:
        tmp = data[data.raceId==raceid]
        
    fig.suptitle(f'Grand Prix of {country}, {year}', fontsize=18, color='w')
        
    last_lap = tmp.groupby('DriverName', as_index=False).lap.max()
    order = pd.merge(tmp, last_lap, on=['DriverName', 'lap']).sort_values(by='position').DriverName.values
    
    tmp = tmp[['DriverName', 'lap', 'seconds']].copy()

    tmp = tmp.set_index(['DriverName', 'lap']).unstack().cumsum(axis=1).reset_index()
    tmp = tmp.set_index('DriverName').reindex(order)
    tmp.columns = tmp.columns.get_level_values(1)
    basis = tmp.median()
    
    if colors is None:
        (- (tmp - basis)).T.plot(ax=ax)
    else:
        cols = []
        for driver in tmp.index:
            if driver in colors.keys():
                cols.append(colors[driver])
            else:
                cols.append('#7C7373')
        (- (tmp - basis)).T.plot(ax=ax, color=cols)
    
    ax.set_xlim((tmp.columns[0], tmp.columns[-1]))
    ax.set_xlabel('Lap', fontsize=14)
    ax.set_ylabel('Delta time (seconds)', fontsize=14)
    
    leg = ax.legend(facecolor="#292525", loc='center left', bbox_to_anchor=(1, 0.5))
    for text in leg.get_texts():
        text.set_color("w")
    ax = plot_frame(ax)
    plt.show()

In [ ]:
plot_race(laps, 'Belgium', 2011, colors={'S. Vettel': clrs['Red Bull'], 
                                         'J. Button': clrs['McLaren'], 
                                         'M. Webber': clrs['Red Bull'], 
                                         'F. Alonso': clrs['Ferrari']})

In principle, nothing particularly remarkable happened: Vettel started from Pole Position with his Red Bull and won the race. In practice, however, we know that Vettel lost the lead on lap 1 in favor of Nico Rosberg who then dropped behind, victim of several DRS assisted overtakes. We see Jenson Button starting from p13, suffering front wing damage in turn 1, and then climbing back while fighting Michael Schumacher first and Mark Webber later until the final p3. We see Alonso in his Ferrari starting p8, fighting for p2 with Webber, and then finishing p4. At last, we can also see the safety car being deployed on lap 13 as a consequence of the incident between Hamilton and Kobayashi that recompacted the field.

Speaking of exciting Belgian Grand Prix, we can't ignore the 1998 edition. The massive car pile-up on turn 1 caused a restart with fewer contenders and we can then see the infamous contact between Schumacher and Coulthard later in the race. Moreover, this was also the first victory for the Jordan team.

In [ ]:
plot_race(laps, 'Belgium', 1998, colors={'M. Schumacher': clrs['Ferrari'], 
                                         'D. Coulthard': clrs['McLaren'], 
                                         'D. Hill': clrs['Jordan']})

Another race that got our attention earlier was the 2013 Bahrain Grand Prix, easily dominated by Sebastian Vettel despite losing the lead on turn 1. However, behind him we find a position change in 91% of the laps, making the most exciting race under this aspect available in the data.

In [ ]:
plot_race(laps, 'Bahrain', 2013)

Bahrain is often the theater of great Grand Prix. For example, one year after we saw the great battle between the two Mercedes drivers going back and forth for the entire race.

In [ ]:
plot_race(laps, 'Bahrain', 2014, {'L. Hamilton': clrs['Mercedes'], 
                                  'N. Rosberg': clrs['Mercedes']})

In more recent years, everyone remembers the thrilling German Grand Prix of 2019, with Lewis Hamilton messing up his race with Mercedes, Verstappen holding on to a lead with the continuous weather changes, and Vettel starting last and climbing back to second.

In [ ]:
plot_race(laps, 'Germany', 2019, {'M. Verstappen': clrs['Red Bull'], 
                                  'S. Vettel': clrs['Ferrari'], 
                                  'L. Hamilton': clrs['Mercedes']})

This year, the Italian Gran Prix was one of the most entertaining, where a safety car and a red flag make the huge lead of Hamilton disappear

In [ ]:
plot_race(laps, 'Italy', 2020, raceid=1038, colors={'P. Gasly': clrs['Red Bull'], 
                                                  'C. Sainz': clrs['McLaren'], 
                                                  'L. Hamilton': clrs['Mercedes']})

On the other hand, the most boring race ever was the 2005 USA Grand Prix, with only 6 cars racing due to technical issues of everyone else. The 2 Ferrari dominated the race and nothing remarkable happened. 

In [ ]:
plot_race(laps, 'USA', 2005, {'M. Schumacher': clrs['Ferrari']})

At last, a race that has been reasonably exciting is the 2015 Singapore Grand Prix. However, what makes this race remarkable is that it is the only race in our data where we had **0 position changes in the top 3 for the entire race**. An unprecedented record that got matched this year and a testament on how hard is to overtake in Singapore when you have a decent pit stop strategy.

In [ ]:
plot_race(laps, 'Singapore', 2015)

The 2020 race that matched this record was in Belgium

In [ ]:
plot_race(laps, 'Belgium', 2020, raceid=None)

# The absence of competition

It is undeniable that the major factor of Mercedes' dominance in the past 7 years is due to the incredible work of team and drivers. However, the 2020 season may have suffered of the drop in perfomance of the main competitor: Ferrari.

Before Red Bull fans start contesting the statement above, let's see how can we see this in the data

* For each Grand Prix, we exclude the top 5% (fast laps attempts) and the bottom 20% (incidents, pit stops, safety cars) of the laps of each team.
* For each Grand Prix, we normalize the data to be between 0 and 1 to have a comparable scale
* For each season, we are going to see how the pace of the top 3 teams evolved by observing both the distribution of lap times and the median values (being the distribution fairly skewed, the median should be a better descriptor)

In [ ]:
teams = ['mercedes', 'red_bull', 'ferrari']

laps20 = laps[(laps.year>2016)].copy()

quantiles = laps20.groupby(['raceId', 
                            'constructorRef']).seconds.quantile([.05, .80]).reset_index().set_index(['level_2', 
                                                                                                    'raceId', 
                                                                                                    'constructorRef']).unstack(0).reset_index()
laps20 = pd.merge(laps20, quantiles, on=['raceId', 'constructorRef'])

laps20 = laps20[(laps20.seconds >= laps20[('seconds', 0.05)]) & (laps20.seconds <= laps20[('seconds', 0.8)])].copy()

del laps20[('seconds', 0.05)]
del laps20[('seconds', 0.8)]

laps20['min_lap'] = laps20.groupby('raceId').seconds.transform('min')
laps20['max_lap'] = laps20.groupby('raceId').seconds.transform('max')

laps20['scaled_seconds'] = laps20['seconds'] - laps20['min_lap']
laps20['scaled_seconds'] /= laps20['max_lap']

laps20.head()

In [ ]:
def plot_pace(data, year, adjust=0.97):
    
    tmp = data[data.year==year]
    
    fig, ax = plt.subplots(tmp.raceId.nunique(), 1, figsize=(13, 4*tmp.raceId.nunique()), facecolor='#292525')
    fig.subplots_adjust(top=adjust)
        
    fig.suptitle(f'Pace Comparison {year}', fontsize=18, color='w')
    
    for n_round in np.arange(1, tmp.raceId.nunique()+1):
        sns.distplot(a=tmp[(tmp.constructorRef=='mercedes') & (tmp['round']==n_round)].scaled_seconds, ax=ax[n_round-1], 
                     kde=True, hist=False, kde_kws={'color': clrs['Mercedes']})
        ax[n_round-1].axvline(tmp[(tmp.constructorRef=='mercedes') & (tmp['round']==n_round)].scaled_seconds.median(), 
                              color=clrs['Mercedes'], linestyle='--')
        sns.distplot(a=tmp[(tmp.constructorRef=='ferrari') & (tmp['round']==n_round)].scaled_seconds, ax=ax[n_round-1], 
                     kde=True, hist=False, kde_kws={'color': clrs['Ferrari']})
        ax[n_round-1].axvline(tmp[(tmp.constructorRef=='ferrari') & (tmp['round']==n_round)].scaled_seconds.median(), 
                              color=clrs['Ferrari'], linestyle='--')
        sns.distplot(a=tmp[(tmp.constructorRef=='red_bull') & (tmp['round']==n_round)].scaled_seconds, ax=ax[n_round-1], 
                     kde=True, hist=False, kde_kws={'color': clrs['Red Bull']})
        ax[n_round-1].axvline(tmp[(tmp.constructorRef=='red_bull') & (tmp['round']==n_round)].scaled_seconds.median(), 
                              color=clrs['Red Bull'], linestyle='--')
        ax[n_round-1].set_title(f"Grand Prix of {tmp[tmp['round']==n_round].country.unique()[0]}", color='w', fontsize=14)

    for axes in ax:
        axes = plot_frame(axes)
        axes.set_xlabel('')
    plt.show()

In [ ]:
plot_pace(laps20, 2018)

In [ ]:
plot_pace(laps20, 2020)